# Context Efficiency in Strands Agents

## Overview

Efficient context management is crucial in Agentic AI, even as Large Language Models (LLMs) advance in their ability to process ever-larger context windows. While attention mechanisms like sliding window and block-wise attention have improved handling of larger contexts, these innovations are relatively new and not fully adopted by most mainstream models. The computational complexity of attention—typically O(N^2)—means that doubling the window size multiplies the required processing by four, which highlights the ongoing need for efficiency. Additionally, larger contexts can lead to the "needle in a haystack" problem, where relevant details are easily lost, further emphasizing the value of focused context practices. In this example, we will demonstrate how to wrap existing python modules with the @tool decorator to create tools that work independently of model context and minimize token consumption.

## Agent Details

<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Custom tools created|add, subtract, Pandas Dataframe manipulation       |
|Agent Structure     |Single agent architecture                          |

</div>


## Strands Agents and Context Management

The Strands Agents framework is inherently suited for efficient context management, particularly through structured tool use and agent memory. For example, when processing an Excel spreadsheet, one approach is to have the agent ingest the entire file using dedicated tools for reading and analyzing Excel data. While feasible for smaller tables, this method does not scale well to enterprise scenarios due to context window limitations. Alternatively, chunking and summarizing multiple rows can reduce context demands, but requires extra coding and can still consume significant model context.

## Leveraging Tools with Strands

Strands makes it easy to convert existing Python functions into agent tools, simply by adding the @tool decorator before defining a function. Using open-source libraries, agents can build memory-resident models of data structures, allowing the agent to manipulate and learn from large data sets without overloading model context.

## Implementing Agent Tools

To maximize efficiency, consider wrapping the Python library in a class and exposing relevant functions as tools, each with concise docstrings summarizing their purpose. These descriptions, informed by the library's API and documentation, help the model decide when to apply specific tools, streamlining both context consumption and usability.

This approach demonstrates how the Strands framework enables scalable, context-efficient workflows for agents operating on real-world, data-intensive tasks.

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Any Amazon Bedrock model that supports Tool Use

Let's now install the requirement packages for our agent.

In [ ]:
# installing pre-requisites
!pip install -r requirements.txt

Here is a trivial arithmetic library, where we define two functions "add" and "subtract":

In [ ]:
%%writefile arithmetic.py
"""
Arithmetic Module
"""

def add(a, b):
    return a + b

def subtract(a, b):
    return a - b

if __name__ == "__main__":
    print(f"5 + 3 = {add(5, 3)}")
    print(f"10 - 4 = {subtract(10, 4)}")

Writing arithmetic.py


We can create tools from this library using Strands:

In [ ]:
# Import from arithmetic.py
from arithmetic import *
from strands import tool

@tool
def add_wrapper(a, b):
    """Add a to b"""
    return add(a, b)

@tool
def subtract_wrapper(a, b):
    """Subtract b from a"""
    return subtract(a, b)

The Python module that wraps the library and serves tools to the agent can then be built as an object-oriented module, with a data structure in the module that retains state as long as the module's calling context remains active. The agent can iteratively process tasks and refine its results, according to its instructions, and selectively use tools to provide information to the model without requiring excessive consumption of context. In this example, we create a class that saves the results of all previous operations in a "paper tape" list:

In [ ]:
# Example of a stateful wrapper class

class ArithmeticTools:

    def __init__(self):
        self.paper_tape = []

    @tool
    def add(self, a, b):
        """Add two numbers"""
        result = a + b
        self.paper_tape.append(result)
        return result

    @tool
    def subtract(self, a, b):
        """Subtract b from a"""
        result = a - b
        self.paper_tape.append(result)
        return result

    @tool
    def print_paper_tape(self):
        """Get the list of previously saved values"""
        return self.paper_tape

When working with large datasets, such as a 100KB Excel spreadsheet, storing the entire file in the model's context can consume between 13,000 and 25,000 tokens—quickly consuming capacity of most current LLMs. In contrast, by loading the spreadsheet into a Python class instance as a Pandas data frame, Strands tools can interact with this data structure and supply only the necessary information to the model, reducing context usage and improving efficiency.

While there are already MCP servers for Pandas and for Excel, these servers do not enable all of the functions we require. So we will now create Strands tools from the existing Python Pandas module. Here is the beginning of the pandas_strands_tools.py module. The full file is available in the current directory.

In [ ]:
!head -n 34 pandas_strands_tools.py

Keep in mind that retaining state in the class as in self.paper_tape, or as a global in the Pandas module, limits the agent to a single instance of state. In a different tutorial, we will explore retaining state as part of the agent's context instead, for instances where the agent needs to access multiple instances at once, such as several spreadsheets. Strands' ability to access tool context is unique among frameworks.
Finally, here is a simple agent that uses the Pandas tools we've created to answer questions about a spreadsheet imported to a Pandas dataframe. Try it with the General-Ledger.xslx file, which is a synthetic public domain data set of accounting ledger entries. You can ask questions like, "Which department had the highest expenses?"

In [ ]:
# pandas_agent.py
# interacts with Pandas module wrapped in Strands @tool decorators
#

from strands import Agent
from pandas_strands_tools import pandas_tools

# Create agent with pre-loaded state and tools
agent = Agent(
    tools=pandas_tools
)

if __name__ == "__main__":
    print("\Pandasbot: Ask me about spreadsheets. Type 'exit' to quit.\n")   

    response = agent("List all available pandas tools")
    print(f"\nExcelBot > {response}")

    # Run the agent in a loop for interactive conversation
    while True:
        user_input = input("\nPrompt > ")
        if user_input.lower() == "exit":
            print("Bye.")
            break
        response = agent(user_input)
        print(f"\nExcelBot > {response}")


While this spreadsheet contains 5000 rows of data, the data is processed by the agent, not the model, which drastically reduces context consumption.

## Strands as a Model-First Framework

Strands is designed with a "model-first" philosophy, emphasizing the LLM’s strengths in reasoning and planning actions. This approach allows seamless integration with existing libraries and modules, enabling agents to manipulate large datasets in memory rather than within the model’s limited context window. By equipping the model with purpose-built tools, Strands leverages external resources for data management while letting the model focus on the higher-level logic and decision-making. This method not only enhances scalability, but also makes Strands especially well-suited for enterprise-grade data processing tasks.
In this notebook, you learned how to use an existing Python module as a tool, to optimize consumption of context tokens in the agent's model.